Install all neccessary libraries

In [2]:
import pandas as pd
import numpy as np
import random
from faker import Faker
from datetime import datetime, timedelta

In [3]:
# Initialize
fake = Faker()
np.random.seed(42)
random.seed(42)

In [4]:
# Setting up Parameters
num_users = 500
num_devices = 3
num_transactions = 10000

Creating Devices Dimension

In [5]:
# Device Types & IDs
device_types = ['Mobile', 'Desktop', 'Tablet']
device_ids = [f"DVC{str(i).zfill(2)}" for i in range(1, num_devices + 1)]

# Create dim_devices
df_devices = pd.DataFrame({
    'DeviceID': device_ids,
    'DeviceType': device_types
})

In [6]:
# Preview Device Dimension
display(df_devices.head(10))

,DeviceID,DeviceType
0,DVC01,Mobile
1,DVC02,Desktop
2,DVC03,Tablet


Creating Users Data Dimension

In [7]:
# Create dim_users
user_ids = [f"USER{str(i).zfill(4)}" for i in range(1, num_users + 1)]
account_status_options = ['Active', 'Suspended', 'Closed']
gender_options = ['Male', 'Female', 'Other']

users = []
for uid in user_ids:
    users.append({
        'UserID': uid,
        'Age': random.randint(18, 70),
        'Gender': random.choice(gender_options),
        'AccountStatus': random.choice(account_status_options),
        'JoinDate': fake.date_between(start_date='-5y', end_date='-30d'),
        'UserLocation': fake.city(),
        'IP_Address': fake.ipv4()
    })

df_users = pd.DataFrame(users)

Preview Users Dimension Data

In [8]:
# Preview Device Dimension
display(df_users.head(10))

,UserID,Age,Gender,AccountStatus,JoinDate,UserLocation,IP_Address
0,USER0001,58,Male,Active,2021-01-11,Richardburgh,149.220.86.38
1,USER0002,65,Female,Active,2021-09-06,East Jared,160.239.35.2
2,USER0003,32,Male,Closed,2022-01-22,Lindaview,60.115.196.171
3,USER0004,24,Other,Closed,2020-07-03,East Sara,24.248.50.202
4,USER0005,52,Male,Closed,2022-02-22,Lake Susan,75.52.233.102
5,USER0006,45,Male,Active,2022-03-31,Mitchellland,191.206.157.252
6,USER0007,23,Male,Active,2022-07-26,New Brookeborough,30.44.87.148
7,USER0008,50,Other,Active,2020-06-24,North Danielberg,145.112.6.118
8,USER0009,53,Male,Closed,2021-09-14,South Wayne,44.161.210.43
9,USER0010,59,Other,Closed,2025-02-21,Lake Kevinstad,186.118.13.59


Creatting Transactions Data Dimension

In [12]:
# Payment Methods
payment_methods = ['Credit Card', 'PayPal', 'Bank Transfer']

# Create fact_transactions
transactions = []
fraud_count = 0
for i in range(num_transactions):
    transaction_id = f"TX{str(i).zfill(6)}"
    user_id = random.choice(user_ids)
    device_id = random.choice(device_ids)
    amount = round(np.random.exponential(scale=120), 2)

    # Random datetime in last 5 years
    days_ago = random.randint(0, 5 * 365)
    random_date = datetime.now() - timedelta(days=days_ago,
                                             hours=random.randint(0, 23),
                                             minutes=random.randint(0, 59))
    
    anomaly_score = round(np.random.beta(2, 5), 2)
    payment_method = random.choice(payment_methods)

    # Realistic fraud logic: 9% fraud overall, higher for risky conditions
    base_fraud_chance = 0.09
    fraud_risk = base_fraud_chance

    if amount > 1000:
        fraud_risk += 0.05
    if anomaly_score > 0.8:
        fraud_risk += 0.04
    if payment_method == "PayPal":
        fraud_risk += 0.02

    is_fraud = 1 if random.random() < fraud_risk else 0
    if is_fraud:
        fraud_count += 1

    transactions.append({
        'TransactionID': transaction_id,
        'UserID': user_id,
        'DeviceID': device_id,
        'TransactionAmount': amount,
        'TransactionDate': random_date,
        'PaymentMethod': payment_method,
        'IsFraud': is_fraud,
        'AnomalyScore': anomaly_score
    })

df_facts = pd.DataFrame(transactions)

Preview Transactions Data

In [16]:
display(df_facts.head(10))

,TransactionID,UserID,DeviceID,TransactionAmount,TransactionDate,PaymentMethod,IsFraud,AnomalyScore
0,TX000000,USER0165,DVC02,42.59,2021-02-16 13:24:21.530545,Bank Transfer,0,0.30
1,TX000001,USER0256,DVC02,93.92,2021-01-22 17:09:21.530545,PayPal,0,0.30
2,TX000002,USER0353,DVC02,145.08,2023-06-13 16:41:21.530545,Bank Transfer,1,0.40
3,TX000003,USER0457,DVC03,8.93,2025-03-14 13:57:21.530545,Credit Card,0,0.14
4,TX000004,USER0436,DVC02,281.06,2024-01-23 00:32:21.530545,PayPal,0,0.23
5,TX000005,USER0457,DVC03,146.30,2023-09-22 01:01:21.530545,Credit Card,0,0.51
6,TX000006,USER0284,DVC01,25.18,2024-03-01 08:43:21.530545,PayPal,0,0.28
7,TX000007,USER0171,DVC03,132.15,2023-12-31 06:36:21.530545,PayPal,0,0.65
8,TX000008,USER0415,DVC01,14.00,2024-08-16 20:20:21.531528,Bank Transfer,0,0.18
9,TX000009,USER0418,DVC02,266.53,2021-01-18 21:16:21.531528,PayPal,0,0.21


In [ ]:
# Save to Excel
with pd.ExcelWriter("fraud_model_data.xlsx") as writer:
    df_facts.to_excel(writer, sheet_name="fact_transactions", index=False)
    df_users.to_excel(writer, sheet_name="dim_users", index=False)
    df_devices.to_excel(writer, sheet_name="dim_devices", index=False)